In [43]:
import os
import json
import csv
from dotenv import load_dotenv
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import Qdrant

# Replace these placeholders with your actual Azure OpenAI credentials
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")  # API key
DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")  # The deployment name
AZURE_OPENAI_ENDPOINT = "https://096290-oai.openai.azure.com"
API_VERSION = "2023-05-15"

# Initialize the Azure OpenAI Chat Model
llm = AzureChatOpenAI(
    azure_deployment=DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_version=API_VERSION,
    openai_api_type="azure",  # Specify the API type as 'azure'
    temperature=0.7  # Adjust temperature as per your use case
)

# Define a chat prompt template
prompt_template = ChatPromptTemplate.from_template("{input}")




In [44]:
users_inputs = ["""Hi, we’re planning a family vacation and need some help putting it together. 
                  There will be four of us—two adults and two kids, aged 8 and 12. We’re looking for something relaxing but fun 
                  for the kids, too. Ideally, we’d want a mix of outdoor activities like hiking or water sports and some downtime to 
                  just enjoy the scenery or maybe a nice pool. We’re thinking of traveling sometime in the summer, probably for a week. 
                  If there’s a place with family-friendly attractions or something educational for the kids, that would be great""",    

    """My partner and I are looking to plan a romantic getaway for just the two of us. We’d like something quiet and intimate, 
    maybe with beautiful views, good food, and opportunities to unwind—like spa treatments or a private villa. We’d also love to 
    explore a bit, maybe take a cooking class or do some wine tasting, but nothing too strenuous. We’re thinking of taking this trip 
    in the spring for about 4 or 5 days.""",

    """Hi there! My friends and I are planning an adventure trip, and we’d love your help organizing it. There are five of us, and 
    we’re all pretty active—we’re into things like kayaking, rock climbing, and maybe even some zip-lining. We’re looking for 
    something that gets us outdoors and keeps us moving but also gives us a chance to explore local culture and try some great food.
      We’re thinking about a 10-day trip in the fall""",
]

In [45]:
def write_token_usage_to_csv(response):
    token_usage = response.response_metadata['token_usage']
        # tocken_usage = {'completion_tokens': <output tokens>,
                        #  'prompt_tokens': <input tokens>,
                        #  'total_tokens': ,
                        #  'completion_tokens_details': {'accepted_prediction_tokens': 0,
                        #   'audio_tokens': 0,
                        #   'reasoning_tokens': 0,
                        #   'rejected_prediction_tokens': 0},
                        #  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}
    input_token_counter, output_token_counter = token_usage['prompt_tokens'], token_usage['completion_tokens']
    file_path = 'token_usage.csv'
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([input_token_counter, output_token_counter])

def parse_user_input(user_input):
    """
    Parse user input to extract key features like budget, number of people,
    wanted time, activities, and former trips.
    """
    prompt = PromptTemplate(
        input_variables=["user_input"],
        template="""
        Extract the following information from the user input and return it as a JSON object, if the informtion is not present write "not specified":
        - Budget
        - Number of people
        - Desired duration
        - Preferred activities
        - Previous trips and their feedback

        User input: {user_input}
        """
    )
    formatted_prompt = prompt.format(user_input=user_input)
    messages = [HumanMessage(content=formatted_prompt)]
    response = llm(messages=messages)
    return response



In [46]:
response = parse_user_input(users_inputs[0])
write_token_usage_to_csv(response)

print(response)

content='```json\n{\n  "Budget": "not specified",\n  "Number of people": 4,\n  "Desired duration": "a week",\n  "Preferred activities": "outdoor activities like hiking or water sports, downtime to enjoy the scenery, a nice pool, family-friendly attractions, something educational for the kids",\n  "Previous trips and their feedback": "not specified"\n}\n```' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 193, 'total_tokens': 272, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None} id='run-a8926072-2af2-402b-acc3-88e07ed04662-0'


In [47]:
for item in response:
    print(item)

('content', '```json\n{\n  "Budget": "not specified",\n  "Number of people": 4,\n  "Desired duration": "a week",\n  "Preferred activities": "outdoor activities like hiking or water sports, downtime to enjoy the scenery, a nice pool, family-friendly attractions, something educational for the kids",\n  "Previous trips and their feedback": "not specified"\n}\n```')
('additional_kwargs', {})
('response_metadata', {'token_usage': {'completion_tokens': 79, 'prompt_tokens': 193, 'total_tokens': 272, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_f3927aa00d', 'finish_reason': 'stop', 'logprobs': None})
('type', 'ai')
('name', None)
('id', 'run-a8926072-2af2-402b-acc3-88e07ed04662-0')
('example', False)
('tool_calls', [])
('invalid_tool_calls', [])
('usage_metadata', None)


# Storage Tool - API online data

In [69]:
import os
import random
import uuid
import numpy as np
from qdrant_client.http.models import PointStruct, VectorParams, Distance
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

load_dotenv()
# Qdrant connection parameters
# Replace with your actual Qdrant cluster details and API key if applicable
VECTOR_DB_URL = os.getenv("QDRANT_URL")  # your Qdrant URL
API_KEY = os.getenv("QDRANT_API_KEY")  # if required


# Initialize Qdrant client.
client = QdrantClient(url=VECTOR_DB_URL, api_key=API_KEY)
# client = QdrantClient(url="http://localhost:6333")


In [70]:
collection_name = "local_flights"
vector_dim = 5
client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_dim, distance=Distance.COSINE),
    )
print(f"Collection '{collection_name}' created.")

ResponseHandlingException: [WinError 10061] No connection could be made because the target machine actively refused it

In [67]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

client.create_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'404 page not found\n'

In [ ]:



def create_collection_if_not_exists(collection_name: str, vector_dim: int = 5):
    """
    Create a collection in Qdrant if it does not already exist.
    """
    # try:
    #     # Try to retrieve the collection information.
    #     collection_info = client.get_collection(collection_name=collection_name)
    #     if collection_info:
    #         print(f"Collection '{collection_name}' already exists.")
    #         return
    # except Exception as e:
    #     # If an exception occurs, assume the collection does not exist.
    #     print(f"Collection '{collection_name}' not found. Proceeding to create it.")

    # Create the collection.
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_dim, distance=Distance.COSINE),
    )
    print(f"Collection '{collection_name}' created.")
# Create collections for flights, hotels, and activities.
VECTOR_DIM = 5  # dimension of dummy vector embeddings
create_collection_if_not_exists("flights", VECTOR_DIM)
create_collection_if_not_exists("hotels", VECTOR_DIM)
create_collection_if_not_exists("activities", VECTOR_DIM)

# Helper function to generate a random vector of the specified dimension.
def generate_random_vector(dim: int = VECTOR_DIM):
    return np.random.rand(dim).tolist()

# Generate synthetic flight data.
def generate_flight_data(num_records: int):
    flights = []
    for _ in range(num_records):
        flight = {
            "id": str(uuid.uuid4()),
            "price": round(random.uniform(100, 2000), 2),
            "location": random.choice(["New York", "London", "Paris", "Tokyo", "Sydney"]),
            "company_name": random.choice(["AirwaysX", "SkyHigh", "FlyFast", "JetSet"]),
            "company_rate": round(random.uniform(3.0, 5.0), 1),
            "seat_info": random.choice(["Economy", "Business", "First Class"]),
            "commission": round(random.uniform(10, 200), 2),
            "vector": generate_random_vector()
        }
        flights.append(flight)
    return flights

# Generate synthetic hotel data.
def generate_hotel_data(num_records: int):
    hotels = []
    for _ in range(num_records):
        hotel = {
            "id": str(uuid.uuid4()),
            "price": round(random.uniform(50, 500), 2),
            "location": random.choice(["New York", "London", "Paris", "Tokyo", "Sydney"]),
            "company_name": random.choice(["HotelComfort", "StayEasy", "LuxuryLodge", "BudgetInn"]),
            "company_rate": round(random.uniform(3.0, 5.0), 1),
            "room_info": random.choice(["Single", "Double", "Suite"]),
            "commission": round(random.uniform(5, 100), 2),
            "vector": generate_random_vector()
        }
        hotels.append(hotel)
    return hotels

# Generate synthetic activity data.
def generate_activity_data(num_records: int):
    activities = []
    for _ in range(num_records):
        activity = {
            "id": str(uuid.uuid4()),
            "price": round(random.uniform(20, 300), 2),
            "location": random.choice(["New York", "London", "Paris", "Tokyo", "Sydney"]),
            "company_name": random.choice(["FunTimes", "AdventureX", "CityTours", "ExperienceIt"]),
            "company_rate": round(random.uniform(3.0, 5.0), 1),
            "activity_info": random.choice(["Museum visit", "City tour", "Concert ticket", "Theme park entry"]),
            "commission": round(random.uniform(5, 50), 2),
            "vector": generate_random_vector()
        }
        activities.append(activity)
    return activities

# Generate a fixed number of records for demonstration.
flights = generate_flight_data(10)
hotels = generate_hotel_data(10)
activities = generate_activity_data(10)

# Function to insert data into a specified Qdrant collection.
def insert_data_into_qdrant(collection_name: str, records: list):
    points = []
    for record in records:
        # Create a point where the vector and payload are set.
        point = PointStruct(
            id=record["id"],
            vector=record["vector"],
            payload={key: value for key, value in record.items() if key not in ["id", "vector"]}
        )
        points.append(point)
    # Upsert the points into the collection.
    client.upload_records(collection_name=collection_name, records=points)
    print(f"Inserted {len(points)} points into the '{collection_name}' collection.")

# Insert the synthetic data into their respective Qdrant collections.
insert_data_into_qdrant("flights", flights)
insert_data_into_qdrant("hotels", hotels)
insert_data_into_qdrant("activities", activities)


C:\Users\matan\AppData\Local\Temp\ipykernel_20340\2565171361.py:17: UserWarning: Failed to obtain server version. Unable to check client-server compatibility. Set check_version=False to skip version check.
  client = QdrantClient(url=VECTOR_DB_URL, api_key=API_KEY)
C:\Users\matan\AppData\Local\Temp\ipykernel_20340\2565171361.py:35: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'404 page not found\n'

# Idan Example usage


In [7]:
# Example text completion function
def generate_completion(user_input):
    """
    Generates a completion using the Azure OpenAI Chat API via LangChain.

    Args:
        user_input (str): The input prompt from the user.

    Returns:
        str: The completion response from the model.
    """
    # Render the prompt
    formatted_prompt = prompt_template.format(input=user_input)
    messages = [HumanMessage(content=formatted_prompt)]

    # Generate response
    response = llm(messages=messages)
    return response.content

if __name__ == "__main__":
    # Example usage
    user_prompt = "Write a short poem about the sea."
    completion = generate_completion(user_prompt)
    print("AI Response:", completion)

C:\Users\matan\AppData\Local\Temp\ipykernel_20340\3744933326.py:17: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages=messages)


AI Response: Beneath the sky so vast and free,  
Whispers the soul of the endless sea.  
Its waves, they dance, a timeless tune,  
Under the gaze of a silver moon.  

It carries secrets, old and deep,  
Dreams it cradles, memories it keeps.  
A canvas of blue, where stories flow,  
Eternal tides come, eternal tides go.  


In [17]:

# Micro Agent 1 - Find Destination
def find_destination(customer_profile):
    """
    Query the Qdrant database for destinations matching the customer profile.
    """
    query = f"Find destinations matching this profile: {customer_profile}"
    results = vectorstore.similarity_search(query, k=5)
    return [result['text'] for result in results]

find_destination_tool = Tool(
    name="Find Destination",
    func=find_destination,
    description="Finds suitable travel destinations based on customer profile."
)

# Micro Agent 2 - Trip Matching
def trip_matching(destination, customer_profile):
    """
    Query the Qdrant database for flights, hotels, transportation, and activities for a destination.
    """
    query = f"Find travel options for destination {destination} with profile: {customer_profile}"
    results = vectorstore.similarity_search(query, k=5)
    return [result['text'] for result in results]

trip_matching_tool = Tool(
    name="Trip Matching",
    func=trip_matching,
    description="Matches travel options to customer preferences."
)

# Micro Agent 3 - Commission Optimization
def optimize_commission(travel_options):
    """
    Optimize travel options for maximum commission.
    """
    sorted_options = sorted(travel_options, key=lambda x: x.get("commission_rate", 0), reverse=True)
    return sorted_options[:3]

commission_optimization_tool = Tool(
    name="Commission Optimization",
    func=optimize_commission,
    description="Optimizes travel options for maximum commission."
)

# Output Formatter
def format_output(options):
    """
    Format the final output with organized and concise options.
    """
    return json.dumps({"recommended_options": options}, indent=2)

output_formatter_tool = Tool(
    name="Output Formatter",
    func=format_output,
    description="Formats the final output for user readability."
)

# Define the tools
tools = [
    find_destination_tool,
    trip_matching_tool,
    commission_optimization_tool,
    output_formatter_tool,
]

# Initialize the agent
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)



In [ ]:

# Main Workflow

def travelwise_agent():
    print("Welcome to TravelWise! I am here to assist with your travel planning.")
    while True:
        user_input = input("Please describe your travel preferences (or type 'exit' to quit):\n")
        if user_input.lower() == "exit":
            print("Thank you for using TravelWise. Have a great day!")
            break

        # Parse user input
        customer_profile = parse_user_input(user_input)

        # Find destinations
        destinations = find_destination(customer_profile)

        # Match trips for each destination
        all_options = []
        for destination in destinations:
            options = trip_matching(destination, customer_profile)
            all_options.extend(options)

        # Optimize for commission
        optimized_options = optimize_commission(all_options)

        # Format output
        final_output = format_output(optimized_options)
        print(f"\nHere are your travel options:\n{final_output}\n")


In [ ]:
# Main execution
if __name__ == "__main__":
    travelwise_agent()